In [21]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Sequential, optimizers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
!pip install d2l==0.17.0

#Model 1

In [20]:
(X_train_val, y_train_val), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
print('train images:', X_train_val.shape)
print('test images:', X_test.shape)

y_train_val = tf.keras.utils.to_categorical(y_train_val, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)
print('train labels:', y_train_val.shape)
print('test labels:', y_test.shape)

train images: (50000, 32, 32, 3)
test images: (10000, 32, 32, 3)
train labels: (50000, 10)
test labels: (10000, 10)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((45000, 32, 32, 3), (5000, 32, 32, 3), (45000, 10), (5000, 10))

In [23]:
def bn_relu_convolution(x, nb_channels, dropout_rate=None, bottleneck=False, weight_decay=1e-4):

    if bottleneck:
        bottleneckWidth = 4
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(nb_channels * bottleneckWidth, (1, 1),
                          kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)
        # Dropout
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

    # BN-ReLU-Conv
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(nb_channels, (3, 3), padding='same')(x)

    # Dropout
    if dropout_rate:
        x = layers.Dropout(dropout_rate)(x)

    return x

In [24]:
def bn_relu_transition(x, nb_channels, dropout_rate=None, compression=1.0, weight_decay=1e-4):

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu',)(x)
    x = layers.Convolution2D(int(nb_channels * compression), (1, 1), padding='same',
                             kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)

    if dropout_rate:
        x = layers.Dropout(dropout_rate)(x)

    x = layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

In [25]:
def dense_block(x, num_layers, nb_channels, growth_rate, dropout_rate=None, bottleneck=False,
                    weight_decay=1e-4):

    for i in range(num_layers):
        cb = bn_relu_convolution(x, growth_rate, dropout_rate, 
                                 bottleneck)
        nb_channels += growth_rate
        x = layers.concatenate([cb, x])
    return x, nb_channels

In [26]:
 def DenseNet(input_shape, dense_blocks, dense_layers, growth_rate, compression, bottleneck, 
                     weight_decay, dropout_rate, num_classes, ):

        img_input = layers.Input(shape=input_shape, name = 'img_input')
        nb_channels = growth_rate


        x = layers.Conv2D(2 * growth_rate, (3, 3), padding='same', strides=(1, 1),name='input_conv', 
                          kernel_regularizer= tf.keras.regularizers.l2(weight_decay))(img_input)


        for block in range(dense_blocks - 1):

            x, nb_channels = dense_block(x, dense_layers[block], nb_channels, growth_rate,
                                     dropout_rate, bottleneck, weight_decay) 

            x = bn_relu_transition(x, nb_channels, dropout_rate, compression, weight_decay)
            nb_channels = int(nb_channels * compression)


        x, nb_channels = dense_block(x, dense_layers[-1], nb_channels,
                                          growth_rate, dropout_rate, weight_decay)

        x = layers.BatchNormalization(name = 'prediction_bn')(x)
        x = layers.Activation('relu',  name = 'prediction_relu', )(x)
        x = layers.GlobalAveragePooling2D( name = 'prediction_pool', )(x)
        prediction = layers.Dense(num_classes, name = 'prediction_dense', activation='softmax')(x)

        return tf.keras.Model(inputs=img_input, outputs=prediction, name='densenet')

In [27]:
dense_net = DenseNet(input_shape = (32,32,3), dense_blocks = 3, dense_layers = [16]*3,
                     growth_rate = 12, compression = 0.5, num_classes = 10, bottleneck = True, 
                     dropout_rate = None, weight_decay = 1e-5)

In [28]:
class DenseNet(object):
    
    def __init__(self,input_shape=None, dense_blocks=3, dense_layers=-1, growth_rate=12, num_classes=None,
                 dropout_rate=None, bottleneck=False, compression=1.0, weight_decay=1e-4, depth=40):
        
        # Parameters Check
        if num_classes == None:
            raise Exception(
                'Please define number of classes (e.g. num_classes=10). This is required to create .')

        if compression <= 0.0 or compression > 1.0:
            raise Exception('Compression have to be a value between 0.0 and 1.0.')

        if type(dense_layers) is list:
            if len(dense_layers) != dense_blocks:
                raise AssertionError('Number of dense blocks have to be same length to specified layers')
        elif dense_layers == -1:
            dense_layers = int((depth - 4) / 3)
            if bottleneck:
                dense_layers = int(dense_layers / 2)
            dense_layers = [dense_layers for _ in range(dense_blocks)]
        else:
            dense_layers = [dense_layers for _ in range(dense_blocks)]

        self.dense_blocks = dense_blocks
        self.dense_layers = dense_layers
        self.input_shape = input_shape
        self.growth_rate = growth_rate
        self.weight_decay = weight_decay
        self.dropout_rate = dropout_rate
        self.bottleneck = bottleneck
        self.compression = compression
        self.num_classes = num_classes
        
        
    def build_model(self):
        """
        Build the model
        Returns: tf Keras Model instance
        """
        if self.bottleneck:
            print('Creating DenseNet with Bottlenecks')
        else:
            print('Creating DenseNet without Bottlenecks')
        print('-' * 50)
        print('No.of. dense blocks: %s' % self.dense_blocks)
        print('Layers per dense block: %s' % self.dense_layers)
        print('-'* 50)


        img_input = layers.Input(shape = self.input_shape, name = 'img_input')
        nb_channels = self.growth_rate


        x = layers.Conv2D(2 * self.growth_rate, (3, 3), padding='same', strides=(1, 1),name='input_conv', 
                          kernel_regularizer= tf.keras.regularizers.l2(self.weight_decay))(img_input)


        for block in range(self.dense_blocks - 1):

            x, nb_channels = self.dense_block(x, self.dense_layers[block], nb_channels, self.growth_rate,
                                      self.dropout_rate, self.bottleneck, self.weight_decay) 

            x = self.bn_relu_transition(x, nb_channels, self.dropout_rate, 
                                        self.compression, self.weight_decay) # 1 conv layer
            nb_channels = int(nb_channels * self.compression)


        x, nb_channels = self.dense_block(x, self.dense_layers[-1], nb_channels,
                                          self.growth_rate, self.dropout_rate, self.weight_decay)
        

        x = layers.BatchNormalization(name = 'prediction_bn')(x)
        x = layers.Activation('relu',  name = 'prediction_relu', )(x)
        x = layers.GlobalAveragePooling2D( name = 'prediction_pool', )(x)
        prediction = layers.Dense(self.num_classes, name = 'prediction_dense', activation='softmax')(x)

        return tf.keras.Model(inputs=img_input, outputs=prediction, name='DenseNet')
        
        
    def dense_block(self, x, num_layers, nb_channels, growth_rate, dropout_rate=None, bottleneck=False,
                    weight_decay=1e-4):

        for i in range(num_layers):
            cb = self.bn_relu_convolution(x, growth_rate, dropout_rate, 
                                     bottleneck) # 1 conv if bottleneck = 0 else 2 conv if bottleneck = 1
            nb_channels += growth_rate
            x = layers.concatenate([cb, x])
        return x, nb_channels

        
    def bn_relu_convolution(self, x, nb_channels, dropout_rate=None, bottleneck=False, weight_decay=1e-4):

        if bottleneck:
            bottleneckWidth = 4
            x = layers.BatchNormalization()(x)
            x = layers.Activation('relu')(x)
            x = layers.Conv2D(nb_channels * bottleneckWidth, (1, 1),
                              kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)
            # Dropout
            if dropout_rate:
                x = layers.Dropout(dropout_rate)(x)

        # BN-ReLU-Conv
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(nb_channels, (3, 3), padding='same')(x)

        # Dropout
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

        return x

    def bn_relu_transition(self, x, nb_channels, dropout_rate=None, compression=1.0, weight_decay=1e-4):
        """
        Creates a transition layer between dense blocks as transition, which do convolution and pooling.
        Works as downsampling.
        """

        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu',)(x)
        x = layers.Convolution2D(int(nb_channels * compression), (1, 1), padding='same',
                                 kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)

        # Adding dropout
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

        x = layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
        return x

In [29]:
dense_net = DenseNet(input_shape = (32,32,3), dense_blocks = 3, dense_layers = [16]*3,
                     growth_rate = 12, compression = 0.5, num_classes = 10, bottleneck = True, 
                     dropout_rate = None, weight_decay = 1e-6).build_model()

Creating DenseNet with Bottlenecks
--------------------------------------------------
No.of. dense blocks: 3
Layers per dense block: [16, 16, 16]
--------------------------------------------------


In [ ]:
dense_net.summary()

Model: "DenseNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 input_conv (Conv2D)            (None, 32, 32, 24)   672         ['img_input[0][0]']              
                                                                                                  
 batch_normalization_122 (Batch  (None, 32, 32, 24)  96          ['input_conv[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_98 (Activation)     (None, 32, 32, 24)   0           ['batch_normalization_122[

In [ ]:
from tensorflow.keras.callbacks import *

In [30]:
dense_net.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'],)

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, vertical_flip=True)
train_datagen.fit(X_train)
train_data = train_datagen.flow(X_train, y_train, batch_size = 200)

In [32]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_datagen.fit(X_val)
val_data = val_datagen.flow(X_val, y_val, batch_size = 200)

In [ ]:
res_history = dense_net.fit(train_data, epochs = 100,validation_data = (X_val/255.,y_val))

Epoch 1/100
225/225 [==============================] - 4400s 20s/step - loss: 1.8231 - accuracy: 0.4510 - val_loss: 3.1122 - val_accuracy: 0.1956
Epoch 2/100
225/225 [==============================] - 4426s 20s/step - loss: 1.4091 - accuracy: 0.5951 - val_loss: 2.6801 - val_accuracy: 0.3290
Epoch 3/100
192/225 [========================>.....] - ETA: 10:40 - loss: 1.2296 - accuracy: 0.6517

#Model 2

In [2]:
!pip install d2l==0.17.0

In [3]:
import tensorflow as tf
from d2l import tensorflow as d2l

class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels):
        super(ConvBlock, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.conv = tf.keras.layers.Conv2D(filters=num_channels,
                                           kernel_size=(3, 3), padding='same')

        self.listLayers = [self.bn, self.relu, self.conv]

    def call(self, x):
        y = x
        for layer in self.listLayers.layers:
            y = layer(y)
        y = tf.keras.layers.concatenate([x, y], axis=-1)
        return y

In [4]:
class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_convs, num_channels):
        super(DenseBlock, self).__init__()
        self.listLayers = []
        for _ in range(num_convs):
            self.listLayers.append(ConvBlock(num_channels))

    def call(self, x):
        for layer in self.listLayers.layers:
            x = layer(x)
        return x

In [5]:
blk = DenseBlock(2, 10)
X = tf.random.uniform((4, 8, 8, 3))
Y = blk(X)
Y.shape

TensorShape([4, 8, 8, 23])

In [6]:
class TransitionBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels, **kwargs):
        super(TransitionBlock, self).__init__(**kwargs)
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.conv = tf.keras.layers.Conv2D(num_channels, kernel_size=1)
        self.avg_pool = tf.keras.layers.AvgPool2D(pool_size=2, strides=2)

    def call(self, x):
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.conv(x)
        return self.avg_pool(x)

In [7]:
blk = TransitionBlock(10)
blk(Y).shape

TensorShape([4, 4, 4, 10])

In [8]:
def block_1():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

In [9]:
def block_2():
    net = block_1()
    # `num_channels`: the current number of channels
    num_channels, growth_rate = 64, 32
    num_convs_in_dense_blocks = [4, 4, 4, 4]

    for i, num_convs in enumerate(num_convs_in_dense_blocks):
        net.add(DenseBlock(num_convs, growth_rate))
        # This is the number of output channels in the previous dense block
        num_channels += num_convs * growth_rate
        # A transition layer that halves the number of channels is added
        # between the dense blocks
        if i != len(num_convs_in_dense_blocks) - 1:
            num_channels //= 2
            net.add(TransitionBlock(num_channels))
    return net

In [10]:
def net():
    net = block_2()
    net.add(tf.keras.layers.BatchNormalization())
    net.add(tf.keras.layers.ReLU())
    net.add(tf.keras.layers.GlobalAvgPool2D())
    net.add(tf.keras.layers.Flatten())
    net.add(tf.keras.layers.Dense(10))
    return net

In [11]:
model = net()

In [12]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f4b25ff6a10>>

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
from keras import datasets

In [17]:
(X_train,y_train),(X_test,y_test)=datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [18]:
X_train = X_train/255.0
X_test = X_test/255.0

In [19]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
1563/1563 [==============================] - 258s 163ms/step - loss: 2.4711 - accuracy: 0.1484
Epoch 2/10
1563/1563 [==============================] - 254s 162ms/step - loss: 2.3026 - accuracy: 0.1501
Epoch 3/10
1563/1563 [==============================] - 252s 161ms/step - loss: 2.3026 - accuracy: 0.1486
Epoch 4/10
1563/1563 [==============================] - 252s 162ms/step - loss: 2.3036 - accuracy: 0.1164
Epoch 5/10
 318/1563 [=====>........................] - ETA: 3:19 - loss: 2.3026 - accuracy: 0.0892

KeyboardInterrupt: ignored